## setup

In [2]:
import Interface as I

/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:13: UserWarning: environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. Falling back to default configuration.
  warnings.warn('environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. ' +
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:33: UserWarning: Using file based locking.Please be careful on nfs mounts as file based locking has issues in this case.
  warnings.warn('Using file based locking.'


trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'spock', 'port': 8885, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'localhost', 'port': 6379, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'file'}
Current version: heads/vcbm-2023+0.ge0cff175.dirty
Current pid: 314111


/scratch/visualn/bzfharth/in-silico-install-dir/anaconda_isf3/lib/python3.8/site-packages/pandas_msgpack-0.1.4+14.gfcb0471-py3.8-linux-x86_64.egg/pandas_msgpack/packers.py:78: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Timestamp, Period, Series, DataFrame,  # noqa
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/visualize/cell_morphology_visualizer.py:29: ImportWarning: Dash could not be imported. Interactive visualisations will not work.
  warnings.warn(


Loading mechanisms:
setting up local multiprocessing framework ... done


Loaded modules with __version__ attribute are:
IPython: 7.19.0, Interface: heads/vcbm-2023+0.ge0cff175.dirty, PIL: 8.2.0, _csv: 1.0, _ctypes: 1.1.0, _curses: b'2.2', _decimal: 1.70, argparse: 1.1, backcall: 0.2.0, blosc: 1.10.2, bluepyopt: 1.9.126, bottleneck: 1.3.2, cffi: 1.14.3, click: 7.1.2, cloudpickle: 1.6.0, colorama: 0.4.4, csv: 1.0, ctypes: 1.1.0, cycler: 0.10.0, cytoolz: 0.11.0, dask: 2.30.0, dateutil: 2.8.1, deap: 1.3, decimal: 1.70, decorator: 4.4.2, distributed: 2.30.1, distutils: 3.8.5, fsspec: 0.8.3, future: 0.18.2, ipykernel: 5.3.4, ipython_genutils: 0.2.0, ipywidgets: 7.5.1, jedi: 0.17.1, jinja2: 2.11.2, joblib: 0.17.0, json: 2.0.9, jupyter_client: 6.1.7, jupyter_core: 4.6.3, kiwisolver: 1.3.0, logging: 0.5.1.2, markupsafe: 1.1.1, matplotlib: 3.3.2, mkl: 2.3.0, neuron: 7.8.2, numexpr: 2.7.1, numpy: 1.19.2, pandas: 1.1.3, pandas_msgpack: 0.1.4+14.gfcb0471.dirty, parameters: 0.2.1, parso: 0.7.0, pas

/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/Interface.py:190: UserWarning:

The source folder has uncommited changes!



In [3]:
postfix = "_2023-06-22"
mdb = I.ModelDataBase('/scratch/visualn/bzfharth/simulation_data'+postfix)
print(mdb.keys())

['89', '90', 'WR69', '88', '93', '84', '87', 'WR64', '85', '86', '91', 'WR71', 'examplary_models', 'df_all', 'run_gillis_protocol_v1', 'all_models_py3', 'run_gillis_protocol_v1__all_models_genetic_py3', 'all_models_genetic_py3', 'all_models_genetic_py2_1', 'all_models_genetic_py2_2', 'all_models_RW_BAC2step_seed_11_filtered', 'all_models_RW_BAC2step_seed_17_filtered', 'all_models_RW_BAC2step_seed_1_filtered']


In [5]:
import os
import shutil

def makeCleanDir(dirname):
    if(os.path.exists(dirname)):
        shutil.rmtree(dirname, ignore_errors=False, onerror=None)
    os.mkdir(dirname)

selected_dataset = 'all_models_RW_BAC2step_seed_1_filtered'
csv_outfolder = os.path.join('/scratch/visualn/bzfharth/simulation_data_postprocessed', selected_dataset+postfix)

makeCleanDir(csv_outfolder)

In [6]:
# init
import pickle
import biophysics_fitting.hay_evaluation
import project_specific_ipynb_code.biophysical_models
from project_specific_ipynb_code.biophysical_models import get_model_ddf_from_encapsulating_mdb, objectives_BAC
from project_specific_ipynb_code.biophysical_models import get_ddf_selected
from project_specific_ipynb_code.hot_zone import get_cell_object_from_hoc
from project_specific_ipynb_code.biophysical_models import min_max_plot, normalize_params
from project_specific_ipynb_code.biophysical_models import CurrentAnalysis
from project_specific_ipynb_code.biophysical_models import modify_simulator_to_record_apical_dendrite_conductances

In [7]:
# init
objectives_BAC = project_specific_ipynb_code.biophysical_models.objectives_BAC
objectives_step = project_specific_ipynb_code.biophysical_models.objectives_step
objectives_2BAC = ['1BAC_APheight', '1BAC_ISI', '1BAC_ahpdepth', '1BAC_caSpike_height', '1BAC_caSpike_width', '1BAC_spikecount', '2BAC_APheight', '2BAC_ISI', '2BAC_ahpdepth', '2BAC_caSpike_height', '2BAC_caSpike_width', '2BAC_spikecount', 'bAP_APheight', 'bAP_APwidth', 'bAP_att2', 'bAP_att3', 'bAP_spikecount']

In [8]:
# they are the same for all optimizations with the exception of CaDynamics_E2 (py2 simulations) 
# and CaDynamics_E2_v2 (py3 simulations)

params_py2 = ['ephys.CaDynamics_E2.apic.decay',
       'ephys.CaDynamics_E2.apic.gamma', 'ephys.CaDynamics_E2.axon.decay',
       'ephys.CaDynamics_E2.axon.gamma', 'ephys.CaDynamics_E2.soma.decay',
       'ephys.CaDynamics_E2.soma.gamma', 'ephys.Ca_HVA.apic.gCa_HVAbar',
       'ephys.Ca_HVA.axon.gCa_HVAbar', 'ephys.Ca_HVA.soma.gCa_HVAbar',
       'ephys.Ca_LVAst.apic.gCa_LVAstbar', 'ephys.Ca_LVAst.axon.gCa_LVAstbar',
       'ephys.Ca_LVAst.soma.gCa_LVAstbar', 'ephys.Im.apic.gImbar',
       'ephys.K_Pst.axon.gK_Pstbar', 'ephys.K_Pst.soma.gK_Pstbar',
       'ephys.K_Tst.axon.gK_Tstbar', 'ephys.K_Tst.soma.gK_Tstbar',
       'ephys.NaTa_t.apic.gNaTa_tbar', 'ephys.NaTa_t.axon.gNaTa_tbar',
       'ephys.NaTa_t.soma.gNaTa_tbar', 'ephys.Nap_Et2.axon.gNap_Et2bar',
       'ephys.Nap_Et2.soma.gNap_Et2bar', 'ephys.SK_E2.apic.gSK_E2bar',
       'ephys.SK_E2.axon.gSK_E2bar', 'ephys.SK_E2.soma.gSK_E2bar',
       'ephys.SKv3_1.apic.gSKv3_1bar', 'ephys.SKv3_1.apic.offset',
       'ephys.SKv3_1.apic.slope', 'ephys.SKv3_1.axon.gSKv3_1bar',
       'ephys.SKv3_1.soma.gSKv3_1bar', 'ephys.none.apic.g_pas',
       'ephys.none.axon.g_pas', 'ephys.none.dend.g_pas',
       'ephys.none.soma.g_pas', 'scale_apical.scale']

params_py3 = [p.replace('ephys.CaDynamics_E2', 'ephys.CaDynamics_E2_v2') for p in params_py2]

In [30]:
folder = mdb[selected_dataset]
folder

'/scratch/visualn/bzfharth/simulation_data_2023-06-22/all_models_rw_bac2step_seed_1_filtered_rqv6y_nj_'

In [31]:
import pandas_msgpack
import glob

files = glob.glob(os.path.join(folder, "pandas_to_msgpack_*"))
files_dict = {}
for file in files:
    idx = int(file.split("_")[-1])
    files_dict[idx] = file

file_indices = sorted(files_dict.keys())

df_0 = pandas_msgpack.read_msgpack(files_dict[0])    
print(df_0.shape)

(62081, 406)


In [32]:
feature_names = set()
blocked = set(['scale_apical.scale', 'model_id', 'type_',
       'morphology', 'lv', 'seed', 'max_', 'gen',
       'hot_zone.outsidescale_sections', 'BAC.hay_measure.recSite',
       'BAC.stim.dist', 'bAP.hay_measure.recSite1', 'bAP.hay_measure.recSite2',
       'hot_zone.min_', 'hot_zone.max_', 'morphology.filename', 'sort_column'])
for col_name in df_0.columns:
    if("ephys" not in col_name):
        feature_names.add(col_name)

feature_names = sorted(feature_names-blocked)
feature_names

['1BAC.check_last_spike_before_deadline',
 '1BAC.check_max_prestim_dendrite_depo',
 '1BAC.check_minspikenum',
 '1BAC.check_no_spike_before_stimulus',
 '1BAC.check_returning_to_rest',
 '1BAC.err',
 '1BAC_APheight',
 '1BAC_APheight.check_1AP',
 '1BAC_APheight.normalized',
 '1BAC_APheight.raw',
 '1BAC_ISI',
 '1BAC_ISI.check_2_or_3_APs',
 '1BAC_ISI.check_repolarization',
 '1BAC_ISI.normalized',
 '1BAC_ISI.raw',
 '1BAC_ahpdepth',
 '1BAC_ahpdepth.check_2AP',
 '1BAC_ahpdepth.normalized',
 '1BAC_ahpdepth.raw',
 '1BAC_caSpike_height',
 '1BAC_caSpike_height.check_1_Ca_AP',
 '1BAC_caSpike_height.check_>=2_Na_AP',
 '1BAC_caSpike_height.check_ca_max_after_nth_somatic_spike',
 '1BAC_caSpike_height.normalized',
 '1BAC_caSpike_height.raw',
 '1BAC_caSpike_width',
 '1BAC_caSpike_width.check_1_Ca_AP',
 '1BAC_caSpike_width.normalized',
 '1BAC_caSpike_width.raw',
 '1BAC_spikecount',
 '1BAC_spikecount.normalized',
 '1BAC_spikecount.raw',
 '2BAC.check_last_spike_before_deadline',
 '2BAC.check_max_prestim_den

In [33]:
for file_idx in file_indices[0:4]:
    df = pandas_msgpack.read_msgpack(files_dict[file_idx])    
    print(df.shape)
    
    parameters = df[params_py3]
    parameters.to_csv(os.path.join(csv_outfolder, "parameters_{}.csv".format(file_idx)))
    
    features = df[feature_names]
    features.to_csv(os.path.join(csv_outfolder, "features_{}.csv".format(file_idx)))

(62081, 406)
(74981, 406)
(69881, 406)
(67881, 406)


In [34]:
features

,1BAC.check_last_spike_before_deadline,1BAC.check_max_prestim_dendrite_depo,1BAC.check_minspikenum,1BAC.check_no_spike_before_stimulus,1BAC.check_returning_to_rest,1BAC.err,1BAC_APheight,1BAC_APheight.check_1AP,1BAC_APheight.normalized,1BAC_APheight.raw,...,mf1,mf2,mf3,record_range_vars.distances,sAHPd1,sAHPd2,sAHPd3,sAHPt1,sAHPt2,sAHPt3
0,True,NaN,True,True,True,238.386665,1.225208,True,1.225208,"[31.295794738561536, 23.022527099556157, 14.89...",...,1.136364,3.571429,3.375034,NaN,0.764935,0.153647,0.148135,1.272216,1.607719,3.676017
1,True,NaN,True,True,True,238.446578,1.373930,True,1.373930,"[31.3175930059544, 22.760408645223272, 12.9482...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,NaN,True,True,True,238.367746,1.257204,True,1.257204,"[31.556935917828596, 22.86191608735096, 14.836...",...,1.704545,6.250000,NaN,NaN,0.733369,0.140599,NaN,1.635868,1.825406,NaN
3,True,NaN,True,True,True,238.462815,1.343018,True,1.343018,"[31.461382385837666, 23.14671917730634, 13.169...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,NaN,True,True,True,238.425267,1.295051,True,1.295051,"[31.266000894267073, 22.501196322221954, 14.33...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67876,True,NaN,True,True,True,236.969047,1.244098,True,1.244098,"[33.09720392961779, 21.480078033857808, 17.955...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67877,True,NaN,True,True,True,236.361948,1.218565,True,1.218565,"[32.9410091742538, 21.274469352249312, 18.3880...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67878,True,NaN,True,True,True,236.101271,1.220039,True,1.220039,"[32.84973225156269, 21.356885538314877, 18.192...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67879,True,NaN,True,True,True,236.656292,1.203209,True,1.203209,"[32.981861031503065, 21.12999791515672, 18.803...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
